In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
def format_data(df):
    df.drop(labels=["Name", "Ticket"], axis=1, inplace=True)
    df.Sex.replace(to_replace=["male","female"], value=[1,0], inplace=True)

    # Convert categorical Embarked values into one hot encoding.
    df = df.join(pd.get_dummies(df.Embarked))
    df.drop(labels="Embarked", axis=1, inplace=True)

    # Replace NaNs in Age with mean.
    df.Age.replace(to_replace=np.nan, value=round(df.Age.mean()), inplace=True)

    # Replace NaNs in Fare with mean.
    df.Fare.replace(to_replace=np.nan, value=round(df.Fare.mean()), inplace=True)

    # Replace NaNs in Cabin with 0 and with 1 for anything else.
    df.Cabin.replace(to_replace=np.nan, value=0,inplace=True)
    df.loc[df['Cabin'] != 0] = 1
    
    return df

In [3]:
df_pred = pd.read_csv("test.csv", index_col=0)
df_pred = format_data(df_pred)

In [4]:
tf.reset_default_graph() # Reset default graph if you want to execute this cell multiple times. 

n_inputs = df_pred.shape[1]
n_hidden1 = 10
n_hidden2 = 5
n_outputs = 1

X = tf.placeholder(tf.float32, shape=(None,n_inputs), name="X")
y = tf.placeholder(tf.float32, shape=(None), name="y")

    
# Define network architecture
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                          kernel_initializer = tf.contrib.layers.xavier_initializer(),
                          name="hidden1")

# This is how you can get the weights or any other property saved in the graph variables.
with tf.variable_scope("hidden1", reuse=True):
    weights1 = tf.get_variable("kernel")

hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                          kernel_regularizer=tf.contrib.layers.l2_regularizer(weights1), 
                          name="hidden2")

with tf.variable_scope("hidden2", reuse=True):
    weights2 = tf.get_variable("kernel")
    
hidden3 = tf.layers.dense(hidden2, n_hidden2, activation=tf.nn.relu,
                          kernel_regularizer=tf.contrib.layers.l2_regularizer(weights2), 
                          name="hidden3")

logits = tf.layers.dense(hidden3, n_outputs, name="logits")

with tf.name_scope("pred"):
    #The default threshold is 0.5, rounded off directly
    prediction = tf.round(tf.sigmoid(logits))


# Assign variable initializer
init = tf.global_variables_initializer()

# Assign model saver
restorer = tf.train.Saver()

In [5]:
#restorer = tf.train.Saver()

with tf.Session() as pred:
    init.run()
    restorer.restore(pred, tf.train.latest_checkpoint('./'))
    output = pred.run(prediction , feed_dict={X:df_pred})

INFO:tensorflow:Restoring parameters from ./titanic.ckpt


In [6]:
titanic_submission = pd.DataFrame(index=df_pred.index)

In [7]:
titanic_submission['Survived'] = output

In [8]:
titanic_submission['Survived'] = titanic_submission.Survived.astype('int64')

In [9]:
titanic_submission.to_csv('submission.csv')

In [10]:
df_pred

Pclass  Sex   Age  SibSp  Parch     Fare  Cabin  C  Q  S
PassengerId                                                          
892               3    1  34.5      0      0   7.8292      0  0  1  0
893               3    0  47.0      1      0   7.0000      0  0  0  1
894               2    1  62.0      0      0   9.6875      0  0  1  0
895               3    1  27.0      0      0   8.6625      0  0  0  1
896               3    0  22.0      1      1  12.2875      0  0  0  1
897               3    1  14.0      0      0   9.2250      0  0  0  1
898               3    0  30.0      0      0   7.6292      0  0  1  0
899               2    1  26.0      1      1  29.0000      0  0  0  1
900               3    0  18.0      0      0   7.2292      0  1  0  0
901               3    1  21.0      2      0  24.1500      0  0  0  1
902               3    1  30.0      0      0   7.8958      0  0  0  1
903               1    1  46.0      0      0  26.0000      0  0  0  1
904               1    1   1.0      1      1   1.0000      1  1  1  1
905               2    1  63.0      1      0  26.0000      0  0  0  1
906               1    1   1.0      1      1   1.0000      1  1  1  1
907               2    0  24.0      1      0  27.7208      0  1  0  0
908               2    1  35.0      0      0  12.3500      0  0  1  0
909               3    1  21.0      0      0   7.2250      0  1  0  0
910               3    0  27.0      1      0   7.9250      0  0  0  1
911               3    0  45.0      0      0   7.2250      0  1  0  0
912               1    1  55.0      1      0  59.4000      0  1  0  0
913               3    1   9.0      0      1   3.1708      0  0  0  1
914               1    0  30.0      0      0  31.6833      0  0  0  1
915               1    1  21.0      0      1  61.3792      0  1  0  0
916               1    1   1.0      1      1   1.0000      1  1  1  1
917               3    1  50.0      1      0  14.5000      0  0  0  1
918               1    1   1.0      1      1   1.0000      1  1  1  1
919               3    1  22.5      0      0   7.2250      0  1  0  0
920               1    1   1.0      1      1   1.0000      1  1  1  1
921               3    1  30.0      2      0  21.6792      0  1  0  0
...             ...  ...   ...    ...    ...      ...    ... .. .. ..
1280              3    1  21.0      0      0   7.7500      0  0  1  0
1281              3    1   6.0      3      1  21.0750      0  0  0  1
1282              1    1   1.0      1      1   1.0000      1  1  1  1
1283              1    1   1.0      1      1   1.0000      1  1  1  1
1284              3    1  13.0      0      2  20.2500      0  0  0  1
1285              2    1  47.0      0      0  10.5000      0  0  0  1
1286              3    1  29.0      3      1  22.0250      0  0  0  1
1287              1    1   1.0      1      1   1.0000      1  1  1  1
1288              3    1  24.0      0      0   7.2500      0  0  1  0
1289              1    1   1.0      1      1   1.0000      1  1  1  1
1290              3    1  22.0      0      0   7.7750      0  0  0  1
1291              3    1  31.0      0      0   7.7333      0  0  1  0
1292              1    1   1.0      1      1   1.0000      1  1  1  1
1293              2    1  38.0      1      0  21.0000      0  0  0  1
1294              1    0  22.0      0      1  59.4000      0  1  0  0
1295              1    1  17.0      0      0  47.1000      0  0  0  1
1296              1    1   1.0      1      1   1.0000      1  1  1  1
1297              1    1   1.0      1      1   1.0000      1  1  1  1
1298              2    1  23.0      1      0  10.5000      0  0  0  1
1299              1    1   1.0      1      1   1.0000      1  1  1  1
1300              3    0  30.0      0      0   7.7208      0  0  1  0
1301              3    0   3.0      1      1  13.7750      0  0  0  1
1302              3    0  30.0      0      0   7.7500      0  0  1  0
1303              1    1   1.0      1      1   1.0000      1  1  1  1
1304              3    